Import libraries

In [57]:
from collections import defaultdict, deque

Import datasets

In [58]:
test_order = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13"""

test_pages = """75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

In [59]:
with open('day5input.txt', 'r') as f:
    data=f.read()

Define functions

In [60]:
def make_sequences(list_of_pages):
    sequences = []
    for pages in list_of_pages.splitlines():
        sequences.append(pages.split(","))
    return sequences

def create_order(pairs_list):
    graph = defaultdict(list)
    in_degree = defaultdict(int)
    
    for pair in pairs_list.splitlines():
        a, b = pair.split("|")
        graph[a].append(b)
        in_degree[b] += 1
        if a not in in_degree:
            in_degree[a] = 0
            
    return graph, in_degree


def topological_sort(graph, in_degree):
    queue = deque([node for node in in_degree if in_degree[node] == 0])
    if not queue:
        print("QUEUE IS EMPTY BORDEL DE MERDE PUTAIN")
    topological_order = []
    #print(queue)
    while queue:
        node = queue.popleft()
        topological_order.append(node)
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    return topological_order


def create_subgraph(sequence, global_graph, global_in_degree):
    """Create a subgraph containing ONLY nodes and edges relevant to the given sequence."""
    subgraph = defaultdict(list)
    sub_in_degree = defaultdict(int)
    sequence_set = set(sequence)
    
    for node in sequence_set:
        if node in global_graph:
            for neighbor in global_graph[node]:
                if neighbor in sequence_set:
                    subgraph[node].append(neighbor)
                    sub_in_degree[neighbor] += 1
            if node not in sub_in_degree:
                sub_in_degree[node] = 0

    return subgraph, sub_in_degree

def check_sequence(sequence, valid_order):
    #just slap enumerate on everything and pretend I'm a programmer
    #print(valid_order)
    pos = {node: index for index, node in enumerate(valid_order)}
    #print(pos)
    for i in range(len(sequence) - 1):
        if sequence[i] not in pos:
            continue
        if pos[sequence[i]] > pos[sequence[i + 1]]:
            print(f"😫: {sequence}")
            return False,[]
    
    print(f"👌: {sequence} middle : {sequence[len(sequence)//2]}")
    return True,sequence[len(sequence)//2]

def sort_sequence(sequence, valid_order):
    print(valid_order)
    pos = {node: index for index, node in enumerate(valid_order)}
    print(pos)
    sorted_sequence = sorted(sequence, key=lambda x: pos.get(x, float('inf')))
    
    print(f"og sequence: {sequence}")
    print(f"sorted sequence: {sorted_sequence}")
    return sorted_sequence


Run example

In [61]:
graph, connections = create_order(test_order)
valid_order = topological_sort(graph, connections)
total=0
sequences = make_sequences(test_pages)
for sequence in sequences:
    valid,middle=check_sequence(sequence, valid_order)
    if valid:
        total+=int(middle)
print(f"Test : {total} == 143")

👌: ['75', '47', '61', '53', '29'] middle : 61
👌: ['97', '61', '53', '29', '13'] middle : 53
👌: ['75', '29', '13'] middle : 29
😫: ['75', '97', '47', '61', '53']
😫: ['61', '13', '29']
😫: ['97', '13', '75', '29', '47']
Test : 143 == 143


Run Part 1

In [62]:
ordering,checking=data.split("\n\n")
graph, connections = create_order(ordering)
# for node, edges in graph.items():
#     print(f"Node {node} points to: {edges}")
print(graph)
print(connections)
valid_order = topological_sort(graph, connections)
print("toporder :", valid_order)
total=0
sequences = make_sequences(checking)
for sequence in sequences:
    
    subgraph, sub_in_degree = create_subgraph(sequence, graph, connections)
    valid_order = topological_sort(subgraph, sub_in_degree)

    
    valid, middle = check_sequence(sequence, valid_order)
    print(f"list type: {type(list)}")
    if valid:
        total += int(middle)
    
print(f"Real Input Total: {total}")

defaultdict(<class 'list'>, {'24': ['55', '18', '87', '77', '92', '52', '14', '44', '61', '22', '71', '82', '72', '78', '68', '64', '63', '51', '67', '88', '99', '15', '85', '91'], '38': ['32', '21', '43', '65', '54', '42', '23', '33', '76', '83', '96', '25', '37', '48', '35', '81', '56', '97', '29', '13', '36', '89', '24', '17'], '48': ['51', '92', '14', '33', '83', '23', '78', '55', '81', '22', '77', '89', '25', '24', '21', '99', '36', '17', '96', '68', '56', '52', '15', '88'], '78': ['35', '54', '87', '44', '68', '72', '77', '43', '64', '38', '61', '97', '71', '76', '42', '32', '85', '65', '91', '63', '82', '18', '67', '29'], '72': ['29', '65', '87', '82', '37', '32', '67', '97', '64', '61', '13', '42', '85', '76', '71', '91', '35', '18', '54', '48', '63', '43', '38', '44'], '23': ['56', '51', '81', '83', '25', '87', '96', '68', '92', '99', '22', '14', '89', '72', '77', '17', '24', '55', '52', '15', '78', '88', '36', '21'], '25': ['14', '18', '55', '22', '15', '77', '17', '72', '51'

Part 2

In [63]:
ordering,checking=data.split("\n\n")
graph, connections = create_order(ordering)
# for node, edges in graph.items():
#     print(f"Node {node} points to: {edges}")
print(graph)
print(connections)
valid_order = topological_sort(graph, connections)
print("toporder :", valid_order)
total=0
total_missprints=0
sequences = make_sequences(checking)

for sequence in sequences:
    subgraph, sub_in_degree = create_subgraph(sequence, graph, connections)
    valid_order = topological_sort(subgraph, sub_in_degree)
    
    valid, middle = check_sequence(sequence, valid_order)
    if valid:
        total += int(middle)
    else:
        p2_sorted_sequence=sort_sequence(sequence,valid_order)
        total_missprints+=int(p2_sorted_sequence[len(p2_sorted_sequence)//2])
        print(p2_sorted_sequence)
        

print(f"missprints Total: {total_missprints}")

defaultdict(<class 'list'>, {'24': ['55', '18', '87', '77', '92', '52', '14', '44', '61', '22', '71', '82', '72', '78', '68', '64', '63', '51', '67', '88', '99', '15', '85', '91'], '38': ['32', '21', '43', '65', '54', '42', '23', '33', '76', '83', '96', '25', '37', '48', '35', '81', '56', '97', '29', '13', '36', '89', '24', '17'], '48': ['51', '92', '14', '33', '83', '23', '78', '55', '81', '22', '77', '89', '25', '24', '21', '99', '36', '17', '96', '68', '56', '52', '15', '88'], '78': ['35', '54', '87', '44', '68', '72', '77', '43', '64', '38', '61', '97', '71', '76', '42', '32', '85', '65', '91', '63', '82', '18', '67', '29'], '72': ['29', '65', '87', '82', '37', '32', '67', '97', '64', '61', '13', '42', '85', '76', '71', '91', '35', '18', '54', '48', '63', '43', '38', '44'], '23': ['56', '51', '81', '83', '25', '87', '96', '68', '92', '99', '22', '14', '89', '72', '77', '17', '24', '55', '52', '15', '78', '88', '36', '21'], '25': ['14', '18', '55', '22', '15', '77', '17', '72', '51'